# Knihovna Multiprocessing

Knihovna `multiprocessing` poskytuje API, které je podobné API `threading`.

Multiprocessing toho umí mnohem více než si tady stihneme ukázat, viz [dokumentace](https://docs.python.org/3/library/multiprocessing.html).

## Vytváření procesů:
Pro vytvoření procesu použijeme třídu `Process` z knihovny multiprocessing.

Proces vytvoříme pomocí konstruktoru třídy `Process`, která přijímá tyto parametry:
* `target=None` - ukazatel na funkci, která se má provést v procesu
* `name=None` - název procesu
* `args=()`/`kwargs={}` - n-tice/slovník argumentů, které se mají předat funkci



In [ ]:
import multiprocessing

def square(x):
    print(x * x)

numbers = [1, 2, 3, 4, 5]
processes = []

for number in numbers:
    process = multiprocessing.Process(target=square, args=(number,))
    processes.append(process)
    process.start()

for process in processes:
    process.join()


### Processes vs Threads
* Procesy jsou oddělené instance programu.
* Mohou běžet současně na více jádrech procesoru.

In [ ]:
def vypocet():
    a = 0
    for i in range(100000000):
        a += i
    return a

In [ ]:
%time vypocet()

In [ ]:
import threading
import time

zacatek = time.time()

vlakno1 = threading.Thread(target=vypocet)
vlakno2 = threading.Thread(target=vypocet)

vlakno1.start()
vlakno2.start()

vlakno1.join()
vlakno2.join()

konec = time.time()
print("Doba trvání: ", konec - zacatek)


In [ ]:
import multiprocessing
import time

zacatek = time.time()

proces1 = multiprocessing.Process(target=vypocet)
proces2 = multiprocessing.Process(target=vypocet)

proces1.start()
proces2.start()

proces1.join()
proces2.join()

konec = time.time()
print("Doba trvání: ", konec - zacatek)

## Nastavení výstupních dat:

Pro předání vstupních dat lze použít přímo "jakékoliv" objekty Pythonu. Pokud však chceme sesbírat výstupy jednotlivých procesů musíme použít knihovnu multiprocessing a třídy `Queue` případně `Pipe`. Zde je příklad použití Queue:

In [ ]:
from multiprocessing import Process, Queue

def square(x, output_queue):
    output_queue.put(x * x)


numbers = [1, 2, 3, 4, 5]
output_queue = Queue()
processes = []

for number in numbers:
    process = Process(target=square, args=(number, output_queue))
    processes.append(process)
    process.start()

for process in processes:
    process.join()

while not output_queue.empty():
    print(output_queue.get())


## Komunikace mezi procesy:
Pro komunikaci mezi procesy můžeme použít opět `Queue` nebo `Pipe`. 

Takto lze použít `Pipe`:

In [ ]:
from multiprocessing import Process, Pipe

def square(x, conn):
    conn.send(x * x)
    conn.send(x * x * x)
    conn.close()


numbers = [1, 2, 3, 4, 5]
processes = []
parent_conns = []

for number in numbers:
    parent_conn, child_conn = Pipe()
    parent_conns.append(parent_conn)
    process = Process(target=square, args=(number, child_conn))
    processes.append(process)
    process.start()

for process in processes:
    process.join()

for parent_conn in parent_conns:
    print(parent_conn.recv())
    print(parent_conn.recv())
    parent_conn.close()




In [ ]:
from multiprocessing import Process, Pipe
import time

def child_process(conn):
    time.sleep(2)
    conn.send("Hello from child process!")
    conn.close()


parent_conn, child_conn = Pipe()
p = Process(target=child_process, args=(child_conn,))
p.start()

while not parent_conn.poll(0.5):  # Poll with a timeout of 0.5 seconds
    print("No message received yet, waiting...")
message = parent_conn.recv()
print(f"Received message: {message}")

p.join()


Pozor!! Metoda poll vrací True i tehdy pokud druhá strana ukončila spojení (close) a už není žádná zpráva k dispozici.

## Synchronizace mezi procesy (barrier):
Pro synchronizaci procesů můžeme použít třídu Barrier z knihovny multiprocessing:

In [ ]:
from multiprocessing import Process, Barrier
import time

def worker(barrier, id):
    print(f'Worker {id} - before barrier')
    time.sleep(1)
    barrier.wait()
    print(f'Worker {id} - after barrier')


num_workers = 5
barrier = Barrier(num_workers)
processes = [Process(target=worker, args=(barrier, i)) for i in range(num_workers)]

for process in processes:
    process.start()

for process in processes:
    process.join()


## Sdílená data
Pro sdílení dat mezi procesy můžeme použít třídy `Value` a `Array` z knihovny multiprocessing. Případně můžeme použít třídu `Manager`.

In [ ]:
from multiprocessing import Process, Value, Array
import time

def f1(n, a):
    for i in range(len(a)):
        a[i] = -a[i]

    n.value = 1

def f2(n, a):
    while n.value == 0:
        print("Waiting...")
    print(a[:])


num = Value('d', 0.0)
arr = Array('i', range(10))

p2 = Process(target=f2, args=(num, arr))
p2.start()

p1 = Process(target=f1, args=(num, arr))
p1.start()



p1.join()
p2.join()


## Pool procesů
`Pool` je třída v knihovně multiprocessing, která poskytuje jednoduchý způsob pro paralelní zpracování úkolů. `Pool` vytváří skupinu (pool) pracovních procesů a umožňuje automatické rozdělení úkolů mezi tyto procesy.

`Pool` je vhodný pro situace, kdy máme mnoho nezávislých úkolů, které mají být zpracovány paralelně. Nejčastěji se používá s funkcí `map`, která aplikuje danou funkci na každý prvek zadaného seznamu a vrátí výsledky v pořadí, ve kterém byly zadané.


In [ ]:
from multiprocessing import Pool
import os

def square(x):
    # get the process id of the current process
    process_id = os.getpid()
    print(f"Process ID: {process_id} zpracovává číslo {x}\n", end="")
    return x * x


numbers = [i for i in range(10)]
with Pool(processes=4) as pool:
    results = pool.map(square, numbers)
print(results)
